In [ ]:
import os
import pickle
import torch
import torchvision
import numpy as np
from torch.utils.data import DataLoader, ConcatDataset
from torch.utils.data import random_split
from torchvision.utils import make_grid
import torchvision.transforms as T
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

matplotlib.rcParams['figure.facecolor'] = '#ffffff'

random_seed = 42
torch.manual_seed(random_seed);

In [ ]:
# Load the data from the file
with open('./data/reconstructed_train_ds.pickle', "rb") as file:
    train_ds = pickle.load(file)

with open('./data/reconstructed_val_ds.pickle', "rb") as file:
    val_ds = pickle.load(file)

with open('./data/reconstructed_test_ds.pickle', "rb") as file:
    test_ds = pickle.load(file)

In [ ]:
from collections import Counter
train_images = [image for image, _ in train_ds]
val_images = [image for image, _ in val_ds]
test_images = [image for image, _ in test_ds]
train_classes = [label for _, label in train_ds]
val_classes = [label for _, label in val_ds]
test_classes = [label for _, label in test_ds]

train_class_size = Counter(train_classes)
val_class_size = Counter(val_classes)
test_class_size = Counter(test_classes)
all_class_size = train_class_size + val_class_size + test_class_size

print(f'Size of train classes: {train_class_size}')
print(f'Size of validation classes: {val_class_size}')
print(f'Size of test classes: {test_class_size}')
print(f'Size of all classes in train, val, and test sets: {all_class_size}')

In [ ]:
batch_size = 64
num_classes = 2
n_cores = os.cpu_count()

torch.cuda.empty_cache()

train_dl = DataLoader(train_ds, batch_size, shuffle=True, num_workers=int(n_cores/2), pin_memory=True)
val_dl = DataLoader(val_ds, batch_size*2, shuffle=True, num_workers=int(n_cores/2), pin_memory=True)

In [ ]:
stats = (0.5, 0.5, 0.5), (0.5, 0.5, 0.5)

def denorm(img_tensors):
    return img_tensors * stats[1][0] + stats[0][0]

def show_images(images, nmax=64):
    fig, ax = plt.subplots(figsize=(8, 8))
    ax.set_xticks([]); ax.set_yticks([])
    ax.imshow(make_grid(denorm(images.detach()[:nmax]), nrow=8).permute(1, 2, 0))
    #ax.imshow(make_grid(images.detach()[:nmax], nrow=8).permute(1, 2, 0))

def show_batch(dl, nmax=64):
    for images, _ in dl:
        show_images(images, nmax)
        break

In [ ]:
show_batch(train_dl)

In [ ]:
# Move data to GPU
from deviceSelector import DeviceDataLoader, to_device

torch.cuda.empty_cache()
train_dl = DeviceDataLoader(train_dl)
val_dl = DeviceDataLoader(val_dl)

print(f'train dataloader device: {train_dl.device}')
print(f'validation dataloader device: {val_dl.device}')

In [ ]:
# Load the model
from deviceSelector import DeviceDataLoader, to_device
from ResNet9 import ResNet9

num_classes = 2

model = to_device(ResNet9(3,num_classes),device='cuda')
model.load_state_dict(torch.load('./trained_models/ResNet9/resnet9_m19.pth'))

In [ ]:
# Define training parameters
epochs = 15
max_lr = 0.01
grad_clip = 0.1
weight_decay = 1e-4
opt_func = torch.optim.Adam

In [ ]:
from ResNet9 import *

history = [evaluate(model, val_dl)]

%%time
history += fit_one_cycle(epochs, max_lr, model, train_dl, val_dl, 
                             grad_clip=grad_clip, 
                             weight_decay=weight_decay, 
                             opt_func=opt_func)

In [ ]:
from plot_history import *

plot_accuracies(history)

In [ ]:
plot_losses(history)

In [ ]:
plot_lrs(history)

In [ ]:
# Save the model
torch.save(model.state_dict(), './trained_models/ResNet9/resnet9_m19_retrained.pth')

In [ ]:
def predict_image(img, model, device='cuda'):
    xb = to_device(img.unsqueeze(0), device)
    yb = model(xb)
    _, preds  = torch.max(yb, dim=1)
    return preds[0].item()

In [ ]:
from sklearn.metrics import classification_report

def eval_test(test_ds, model, device='cuda'):
    with torch.no_grad():
        correct = 0
        total = 0
        y_true = []
        y_pred = []
        
        for img, label in test_ds:
            xb = to_device(img.unsqueeze(0), device)
            yb = model(xb)
            _, preds  = torch.max(yb, dim=1)
            total += 1
            correct += (preds[0] == label).sum().item()
            predicted=preds[0].to('cpu')
            y_true.append(label)
            y_pred.append(predicted)
            
        print('Test Accuracy: {}%'.format(100 * correct / total))

        # Generate a classification report
        print(classification_report(y_true, y_pred))

In [ ]:
# Evaluate the model on the test dataset
eval_test(test_ds, model)